# Building a rule based chatbot

In [1]:
import nltk
from nltk.chat.util import Chat,reflections

In [2]:
print(Chat)

<class 'nltk.chat.util.Chat'>


In [3]:
reflections

{'i am': 'you are',
 'i was': 'you were',
 'i': 'you',
 "i'm": 'you are',
 "i'd": 'you would',
 "i've": 'you have',
 "i'll": 'you will',
 'my': 'your',
 'you are': 'I am',
 'you were': 'I was',
 "you've": 'I have',
 "you'll": 'I will',
 'your': 'my',
 'yours': 'mine',
 'you': 'me',
 'me': 'you'}

In [4]:
set_pairs = [
    [
        r"my name is(.*)",
        ["hello %1, How are you Today?",]
    ],
    [
        r"hi|hello|hey",
        ["hello","hey there",]
    ],
    [
        r"what is your name",
        ["you can call me chatbot",]
    ],
    [
        r"how are you",
        ["I am fine,thank you ! How can i help you",]
    ],
    [
        r"I am fine ,thank you",
        ["great to hear that how can i help you",]
    ],
    [
        r"how can i help you",
        ["I am looking for online guides and courses to learn data science can you suggest",]
    ],
    [
        r"thanks for suggestion,do they have great author and instructors",
        ["yes,they have world class authors ,that is their strength",]
    ],
    [
        r"(.*)thank you  so much,that was helpful",
        ["I am happy to help","No problem","you are welcome",]
    ],
    [
        r"quit",
        ["bye ,take care.see you soon :)","it was nice talking to you. see you soon :)"]
    ],
]


In [5]:
def chatbot():
    print("Hi i am rule based chatbot! what can i help you with ?")
chatbot()

Hi i am rule based chatbot! what can i help you with ?


In [6]:
chat = Chat(set_pairs,reflections)
print(chat)

In [7]:
chat.converse()
if __name__ ==" __main__":
    chatbot()

>hi
hey there
>quit
it was nice talking to you. see you soon :)


# Building a retrieval based chatbot

In [8]:
import nltk
import numpy as np
import string
import random

import  bs4 as bs
import requests
import re

import warnings
warnings.filterwarnings = False

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\shyam
[nltk_data]     suthar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\shyam
[nltk_data]     suthar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Gathering data from Wikipedia

In [11]:
r = requests.get('https://en.wikipedia.org/wiki/Cuisine')
raw_html = r.text
r.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Cuisine - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"4104e2aa-e09a-4d4c-b93f-a4985967d8b9","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Cuisine","wgTitle":"Cuisine","wgCurRevisionId":1086849835,"wgRevisionId":1086849835,"wgArticleId":6656,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with bare URLs for citations","Articles with bare URLs for citations from February 2022","Articles with short description","Short description matches Wikid

In [12]:
# cleaning up
corpus_html = bs.BeautifulSoup(raw_html)

corpus_paras = corpus_html.find_all('p')
corpus_text = ' '

for para in corpus_paras:
    corpus_text += para.text
    
corpus_text = corpus_text.lower()


In [13]:
corpus_text

' \na cuisine  is a style of cooking characterized by distinctive ingredients, techniques and dishes, and usually associated with a specific culture or geographic region. regional food preparation techniques, customs and ingredients often combine to create dishes unique to a region.[1]\na cuisine is partly determined by ingredients that are available locally or through trade. often particular regional varieties of ingredients are developed and contribute to the cuisine, such as japanese rice in japanese cuisine and new mexico chile in new mexican cuisine.\nreligious food laws can also exercise a strong influence on cuisine, such as hinduism in indian cuisine, sikhism in punjabi cuisine, buddhism in east asian cuisine, islam in middle eastern cuisine, and judaism in jewish and israeli cuisine.\ncuisine is borrowed from the french meaning cooking, culinary art and kitchen.[2]\nsome factors that have an influence on a region\'s cuisine include the area\'s climate, the trade among differen

In [14]:
# getting rid all of the empty spaces and special characters
corpus_text = re.sub(r'\[[0-9]*\]',' ',corpus_text)
corpus_text = re.sub(r'\s+',' ',corpus_text)

In [15]:
corpus_sentence = nltk.sent_tokenize(corpus_text)
corpus_word = nltk.word_tokenize(corpus_text)

In [16]:
corpus_sentence

[' a cuisine is a style of cooking characterized by distinctive ingredients, techniques and dishes, and usually associated with a specific culture or geographic region.',
 'regional food preparation techniques, customs and ingredients often combine to create dishes unique to a region.',
 'a cuisine is partly determined by ingredients that are available locally or through trade.',
 'often particular regional varieties of ingredients are developed and contribute to the cuisine, such as japanese rice in japanese cuisine and new mexico chile in new mexican cuisine.',
 'religious food laws can also exercise a strong influence on cuisine, such as hinduism in indian cuisine, sikhism in punjabi cuisine, buddhism in east asian cuisine, islam in middle eastern cuisine, and judaism in jewish and israeli cuisine.',
 'cuisine is borrowed from the french meaning cooking, culinary art and kitchen.',
 "some factors that have an influence on a region's cuisine include the area's climate, the trade amon

In [17]:
corpus_word

['a',
 'cuisine',
 'is',
 'a',
 'style',
 'of',
 'cooking',
 'characterized',
 'by',
 'distinctive',
 'ingredients',
 ',',
 'techniques',
 'and',
 'dishes',
 ',',
 'and',
 'usually',
 'associated',
 'with',
 'a',
 'specific',
 'culture',
 'or',
 'geographic',
 'region',
 '.',
 'regional',
 'food',
 'preparation',
 'techniques',
 ',',
 'customs',
 'and',
 'ingredients',
 'often',
 'combine',
 'to',
 'create',
 'dishes',
 'unique',
 'to',
 'a',
 'region',
 '.',
 'a',
 'cuisine',
 'is',
 'partly',
 'determined',
 'by',
 'ingredients',
 'that',
 'are',
 'available',
 'locally',
 'or',
 'through',
 'trade',
 '.',
 'often',
 'particular',
 'regional',
 'varieties',
 'of',
 'ingredients',
 'are',
 'developed',
 'and',
 'contribute',
 'to',
 'the',
 'cuisine',
 ',',
 'such',
 'as',
 'japanese',
 'rice',
 'in',
 'japanese',
 'cuisine',
 'and',
 'new',
 'mexico',
 'chile',
 'in',
 'new',
 'mexican',
 'cuisine',
 '.',
 'religious',
 'food',
 'laws',
 'can',
 'also',
 'exercise',
 'a',
 'strong',


## Generating greeting response on predefined set of inputs

In [18]:
greeting_inputs = ("hey","good morning","good evening","morning","evening","hi","wassup","ssup",)
greeting_responses = ["hey","hey how are you","*nods*",'hello',"how you doing","welcome i am good"]

def greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

## preprocessing with punctuation and lemmatizing

In [19]:
wn_lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_corpus(tokens):
    return [wn_lemmatizer.lemmatize(token) for token in tokens]
punct_removal_dict = dict((ord(punctuation),None) for punctuation in string.punctuation)

def get_processed_text(document):
    return lemmatize_corpus(nltk.word_tokenize(document.lower().translate(punct_removal_dict)))

## Language modeling with tf-idf

In [26]:
def respond(user_input):
    '''
    generating response after
    '''
    bot_response = ' '
    corpus_sentence.append(user_input)
    word_vectorizer = TfidfVectorizer(tokenizer = get_processed_text,stop_words = 'english')
    corpus_word_vectors = word_vectorizer.fit_transform(corpus_sentence)
    
    
    cos_sim_vectors = cosine_similarity(corpus_word_vectors[-1],corpus_word_vectors)
    similar_response_idx = cos_sim_vectors.argsort()[0][-2]
    
    match_vector = cos_sim_vectors.flatten()
    match_vector.sort()
    vector_matched = match_vector[-2]
    
    
    if vector_matched == 0:
        bot_response = bot_response +" I am sorry ,what is it"
        return bot_response
        
    else:
        bot_response +corpus_sentence[similar_response_idx]
        return bot_response
        

In [ ]:
chat = True
print("Hello, what do you want to learn about Cuisines today")
while (chat == True):
    user_query = input()
    user_query = user_query.lower()
    if user_query !='quit':
        if user_query=="thanks" or user_query == "Thank you":
            chat = False
        else:
            if greeting_response(user_query) != None:
                print("Cuisinebot :" + greeting_response(user_query))
            else:
                print("Cuisinebot :" ,end='')
                print(respond(user_query))
                corpus_sentence.remove(user_query)
                
    else:
        chat = False
        print("CusineBot : Good Bye")

Hello, what do you want to learn about Cuisines today
hi
Cuisinebot :hey
mexican
Cuisinebot : 
pizza
Cuisinebot :  I am sorry ,what is it
